In [ ]:
import cv2 as cv
from pathlib import Path
import numpy as np



# ─ CONFIG ─────────────────────────────────────────
BASE_DATE  = "16_7_2025"
FOLDER_NAME = "sample_lab"
IMAGE_NAME = "DJI_0374_zoomed_29.jpg"
BASE_PATH  = Path(f"/Users/at/Orbit_Red_Blob/Data/UKE_Plot_14_DD_{BASE_DATE}/{FOLDER_NAME}")

img_bgr = cv.imread(str(BASE_PATH / IMAGE_NAME))
if img_bgr is None:
    raise FileNotFoundError(f"Couldn’t load image: {BASE_PATH/IMAGE_NAME}")



# 1) Convert to LAB
lab = cv.cvtColor(img_bgr, cv.COLOR_BGR2LAB)
L, A, B = cv.split(lab)

# 2) Threshold the ‘a’ channel
#   You may need to tweak the [min, max] for your lighting.
lower_a = 130
upper_a = 140

lower_l = 0
upper_l = 40

lower_a = 130
upper_a = 140

lower_l = 0
upper_l = 255

# lower_b, upper_b = 120, 200     # NEW: add b* to exclude greens/oranges

mask = (
    cv.inRange(L, lower_l, upper_l) &
    cv.inRange(A, lower_a, upper_a) 
    # & cv.inRange(B, lower_b, upper_b)   # include B
)


# 3) Clean up and visualize
mask = cv.morphologyEx(mask, cv.MORPH_OPEN, np.ones((3,3),np.uint8))

red_highlight = img_bgr.copy()
red_highlight[mask>0] = (0,0,255)

# cv.imshow("Red via LAB", red_highlight)
cv.imwrite(f"{BASE_PATH}/output_lab.jpg", red_highlight)
# cv.waitKey(0)


print("a-channel: min =", A.min(), "  max =", A.max())
print("a-channel 95th percentile =", np.percentile(A, 95))
print("a-channel 5th percentile =", np.percentile(A, 5))

print("L 5/50/95th:", np.percentile(L, [5,50,95]))
print("A 5/50/95th:", np.percentile(A, [5,50,95]))
print("B 5/50/95th:", np.percentile(B, [5,50,95]))
